In [37]:
#imports
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import itertools

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auggi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
#Read in Datafile
df = pd.read_csv('IMDBDataset.csv').to_numpy()
#temporary, cuts the dataset to 100 to save time running it when testing
df = df[:500]
size = int(df.size/2)
print(size)

500


In [93]:
#Cleans HTML Input from string
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return TextBlob(cleantext)

# This is how the Naive Bayes classifier expects the input
# removes stopwords from string
# returns a dictionary
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

#Process Input
def findusefulwords(arr, sentiment):
    unique_words = {}
    useful_words = []

    for x in range (arr[0].size):
       unique_words[x] = set(cleanhtml(arr[0][x]).split(' '))

    for x in range (len(unique_words)):
        useful_words.append((create_word_features(unique_words[x]), sentiment))
    return useful_words

In [94]:
pos_reviews = []
neg_reviews = []

for x in range (size):
    if(df[x][1] == 'positive'):
        pos_reviews.append((df[x]))
    elif(df[x][1] == 'negative'):
        neg_reviews.append((df[x]))
pos_reviews = np.array(pos_reviews).T
pos_words = findusefulwords(pos_reviews,"positive")

neg_reviews = np.array(neg_reviews).T
neg_words = findusefulwords(neg_reviews,"negative")

In [95]:
print(len(pos_words))
print(len(neg_words))
print(pos_words[0])
print(neg_words[0])

237
263
({'shows': True, 'show': True, 'hearted': True, 'It': True, 'high': True, 'nasty': True, 'street': True, 'me.The': True, 'Em': True, '1': True, 'face': True, 'unflinching': True, 'mess': True, 'may': True, 'exactly': True, 'Emerald': True, 'lack': True, 'mainstream': True, 'audiences,': True, 'Oswald': True, 'Maximum': True, 'saw': True, 'taste': True, 'happened': True, 'Italians,': True, 'never': True, 'struck': True, 'would': True, 'fact': True, 'inwards,': True, 'ever': True, 'painted': True, "who'll": True, 'become': True, 'section': True, 'Its': True, 'mentioned': True, 'brutality': True, 'ready': True, 'One': True, 'main': True, 'They': True, 'skills': True, 'Oz,': True, 'watching': True, 'reviewers': True, 'fronts': True, 'experience)': True, 'thing': True, 'prison': True, 'away': True, 'Not': True, 'turned': True, 'right': True, 'Muslims,': True, 'violence,': True, 'violence.': True, 'GO.': True, 'sold': True, 'City': True, 'Latinos,': True, 'Oz': True, 'watched': True,

In [96]:
train_set = neg_words[:200] + pos_words[:200]
test_set =  neg_words[200:] + pos_words[200:]
print(len(train_set),  len(test_set))

400 100


In [97]:
#Classifier
classifier = NaiveBayesClassifier.train(train_set)

In [98]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy * 100)

73.0


In [ ]:
#https://www.pythonforengineers.com/build-a-sentiment-analysis-app-with-movie-reviews/